In [4]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path

In [71]:
NotInclude = ['scratchPadText', '']
TimeDwellOrig = 800
TimeFixation = 300

In [6]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [70]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [7]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [8]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [89]:
def phrasesTyped(your_list, timeList, userKeys, stimPhraseList, SubjName):
    # Find entries in scratchpad (end of each trial)
    
    IndexNotInclude = list()
    PhraseList = list()
    TypingSpeed = list()

    # After every complete phrase, the next phrase is empyty ''
    for row_ind in range(0, len(your_list)):
        if 'sa\Test_woChinRest\p1' in SubjName or 'rh\Test_wChinRest\p1' in SubjName:
            if row_ind == len(your_list)-1:
                IndexNotInclude.append(row_ind)
                #print([your_list[row_ind-1][1]])
                #print(your_list[row_ind-1][1])
        if your_list[row_ind][1] == '':
            if row_ind == 1:# first empty phrase needs to be not considered , at least for most of the subjects
                #print(your_list[row_ind-1][1])
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
            #print([your_list[row_ind-1][1]])
    
    #print(len(IndexNotInclude))
    #print('IndexNotInclude', IndexNotInclude)
    
    firstTrial = IndexNotInclude[0]
    # Find the time taken for each phrase  
    # remove the indices that are due to deleting phrase written to make the scrathpad empty
    IndexNotInclude_Difference = [t - s for s, t in zip(IndexNotInclude, IndexNotInclude[1:])]
    #print('IndexNotInclude_Difference', IndexNotInclude_Difference)
    IndexNotInclude = [IndexNotInclude[ind+1] for ind in range(0,len(IndexNotInclude_Difference)) if IndexNotInclude_Difference[ind]>12]
    IndexNotInclude_DifferenceLeft = [IndexNotInclude_Difference[ind] for ind in range(0,len(IndexNotInclude_Difference)) if IndexNotInclude_Difference[ind]>12]

    #print(len(IndexNotInclude_DifferenceLeft))
    #print('IndexNotInclude_DifferenceLeft', IndexNotInclude_DifferenceLeft)
    
    if 'sa\Test_woChinRest\p1' in SubjName:
        IndexNotInclude = IndexNotInclude[1:]
    if 'rh\Test_wChinRest\p2' in SubjName:
        del IndexNotInclude[-2]
        IndexNotInclude.insert(0, firstTrial)
        
    if 'sa\Test_woChinRest\p2' in SubjName:
        IndexNotInclude.insert(0, firstTrial)
        
    #print('length', len(userKeys[1]))
    
    for indPhrase in range(0, len(IndexNotInclude)):
        #print(indPhrase)
        startPhrase = timeList[indPhrase]    
 
        endPhrase = IndexNotInclude[indPhrase]-1
        headStartPhrase,sep,tail = userKeys[startPhrase][0].partition('+')
        headEndPhrase,sep,tail = your_list[endPhrase][0].partition('+')
        timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") + datetime.timedelta(0, 0, float(userKeys[startPhrase][3]))
        phraseTyped = your_list[endPhrase][1]
        #print(userKeys[startPhrase][0], ' to ', your_list[endPhrase][0], phraseTyped)
        PhraseList.append([timeTaken, phraseTyped])
            

    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0
    
    #print(IndexNotInclude)
    
    for indPhrase in range(0, len(IndexNotInclude)):
        timeAdd += PhraseList[indPhrase][0]
        
        characterLen += len(PhraseList[indPhrase][1])
        
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
    
    return characterLen, TimeTaken
    

In [93]:
metricComputed = 'typing_speed'
dataFolderName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
TimeDwellAvg = 0
nSub = 0
TypingSpeedList = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'trial' in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)    
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
            
            if fnmatch.fnmatch(file, 'ReplacementPhraseLog.csv'):
                
                try:
                    fPhraseLog = open(root + '\\' + file)
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the replacement phrase log file')
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            
            a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
            subjName = a.findall(root)[0]
            print(subjName)
            
            if subjName == 'sa\Test_woChinRest\p2':
                userKeys = userKeys[:-1]
                    
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
            
            # fix userKeys due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            #print(scratchPad_new)
            # fix userKeys due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # find start time of typing
            #timeTypingStartEnd = OptiKeyTypingTime(userKeys_new)
            
            
                
            # for some of the subjects, the data was not completely collected
            #if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
            #    del keysSelected[-1]
            
            stimPhraseList = [key[1] for key in phraseLog_new]
            
            timeList = timeTypingStart(userKeys_new)
            #print('timeList', len(timeList))
                
            #print(scratchPad_new)
            
            if 'sa\Test_wChinRest\p1' in subjName:
                characters, timeTypingTotal = phrasesTyped(scratchPad_new, timeList[2:], userKeys_wDwellTime, stimPhraseList, subjName) 
            elif 'sa\Test_woChinRest\p2' in subjName or 'rh\Test_wChinRest\p2' in subjName:
                characters, timeTypingTotal = phrasesTyped(scratchPad_new, timeList, userKeys_wDwellTime, stimPhraseList, subjName) 
            else:
                characters, timeTypingTotal = phrasesTyped(scratchPad_new, timeList[1:], userKeys_wDwellTime, stimPhraseList, subjName) 
            
            if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
                if 'p1' in subjName:
                    characters1 = characters
                    timeTypingTotal1 = timeTypingTotal
                    continue
                else:
                    
                    characters2 = characters
                    timeTypingTotal2 = timeTypingTotal
                
                timeTypingTotal = timeTypingTotal1 + timeTypingTotal2
                characters = characters1 + characters2
                
            Names.append(subjName)
            typingSpeed = characters/(5*timeTypingTotal)
            print(subjName, typingSpeed)
            
            TypingSpeedList.append(typingSpeed)
            #print(TypingSpeedList)
            
            
#if TypingSpeedList:
#    print(Names)
#    AddDataToFile(metricComputed, Names, TypingSpeedList)
            

bh\Test_wChinRest
bh\Test_wChinRest 5.942823882499074
bh\Test_woChinRest
bh\Test_woChinRest 8.120209433141628
ph\Test_wChinRest
ph\Test_wChinRest 6.8318215804901685
ph\Test_woChinRest
ph\Test_woChinRest 7.269472744953302
pt\Test_wChinRest
pt\Test_wChinRest 7.553522208547355
pt\Test_woChinRest
pt\Test_woChinRest 7.761126307729862
rh\Test_wChinRest\p1
rh\Test_wChinRest\p2
rh\Test_wChinRest\p2 6.64046267487131
rh\Test_woChinRest
rh\Test_woChinRest 7.723950996939564
sa\Test_wChinRest
sa\Test_wChinRest 5.789037197184803
sa\Test_woChinRest\p1
sa\Test_woChinRest\p2
sa\Test_woChinRest\p2 4.471890404560554


In [16]:
a = [1, 2, [], 4]
b  = [x for x in a if x != []]

In [17]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileSave = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        fileSave = fileIn
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                print(subjNameame)
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
    

In [ ]:
b